좀 더 자세한 알고리즘 설명은 github의 soy/tutorial/space_correction.ipynb를 참고하면 좋습니다. 이 튜토리얼에서는 사용법을 위주로 설명합니다. 

먼저 soy.git을 이용하기 위하여 sys.path에 폴더 주소를 추가합니다. 

soy.nlp.space의 CountSpace를 import 합니다. 

In [1]:
import sys

sys.path.append('../soy')

In [2]:
from soy.nlp.space import RuleDict, CountSpace
model = CountSpace()

## Training

soy에는 라라랜드 영화의 리뷰 샘플이 학습데이터로 올라가 있습니다. 이 데이터는 다양한 띄어쓰기 패턴을 학습하기에는 적은 양이지만, 빠르게 실습을 하기 위해서 이를 이용해보도록 하겠습니다. 

134963_norm은 리뷰데이터에서 한글만 남겨놓은 텍스트와 평점이 \t으로 구분되어있는 데이터입니다. 이로부터 텍스트만을 가져와서 띄어쓰기수정 모델을 학습합니다. CountSpace는 텍스트만 있는 문서에서 한줄씩 텍스트를 읽으며 띄어쓰기 패턴을 학습하기 때문에 평점을 제거한 텍스트를 tmp 폴더에 저장해두겠습니다. 

for _ in range(3): 은 range를 3번 돌라는 의미이며, 무의미한 변수를 만들기 때문에 under-bar 처리하였습니다. 

print(next(f))를 통하여 한 줄씩 출력이 됩니다. 

In [3]:
with open('../soy/data/naver_movie/comments/134963_norm.txt', encoding='utf-8') as f:
    for _ in range(3):
        print(next(f))

시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다	9

사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고	10

지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다	10



In [4]:
with open('../soy/data/naver_movie/comments/134963_norm.txt', encoding='utf-8') as f:
    docs = [doc.split('\t')[0] for doc in f]

with open('./tmp/134963_norm.txt', 'w', encoding='utf-8') as f:
    for doc in docs:
        f.write('%s\n' % doc)

num_lines는 fname 파일의 num_lines 번째 줄까지만 학습하는 argument 입니다. -1이면 모든 줄을 학습에 이용합니다. 코드가 잘 돌아가는지 디버깅을 할 때에는 적당한 숫자를 넣어주시면 그 줄만큼만 학습을 합니다.

In [5]:
model.train(fname='./tmp/134963_norm.txt', num_lines=-1)

all tags length = 654328 --> 53317, (num_doc = 15602)

json 형식으로 parameters를 저장하도록 json_format=True를 설정한 뒤, tmp 폴더에 모델을 저장해둡니다. 

In [6]:
model.save_model('./tmp/space_model.json', json_format=True)

## Test

테스트를 하기 위해서 아래의 패러메터들의 값을 설정합니다. 

- arguments

    - 4개의 parameter
        - force_abs_threshold: 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅
        - nonspace_threshold : 이 점수 이하일 때만 0으로 태깅
        - space_threshold    : 이 점수 이상일 때만 1로 태깅
        - min_count          : L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실한 정보로 판단, 띄어쓰기 계산 시 무시
        
    - verbose: iteration 마다 띄어쓰기가 어떻게 되고 있는지 확인
    
    - rules  : 점수와 관계없이 반드시 태깅을 먼저 할 (chars, tags)
        

In [7]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

before와 after를 보면 띄어쓰기가 수정되었음을 볼 수 있습니다. 

In [8]:
sent = '이건진짜좋은영화 라라랜드진짜좋은영화'

sent_corrected, tags = model.correct(doc=sent, verbose=verbose, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드진짜 좋은 영화


verbose=True 이면, 각 단계별로 어떻게 띄어쓰기가 수정되었는지 확인할 수 있습니다. 

이 경우에는 대부분의 경우 띄어쓰기가 확실해서 force tagging이 되었습니다. tagg가 None으로 뜨는 경우는, 띄어쓰기 정보가 불확실하여 띄어쓰기를 하지 않은 경우입니다. 

In [9]:
corrected, tags = model.correct(doc=sent, verbose=True, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

Input: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Force tagged (iter=1): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 
Force tagged (iter=2): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 


('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

force_abs_threshold의 값을 좀 더 키우면 force tagging이 적게 일어납니다. 속도가 더 느려집니다. 

In [10]:
model.correct(doc=sent, verbose=True, force_abs_threshold=0.9, nonspace_threshold=nt, space_threshold=st, min_count=mc)

Input: ? ? ? ? ? ? ? 1 ? ? ? ? ? ? ? ? ? 1 
이건진짜좋은영화 라라랜드진짜좋은영화
Force tagged (iter=1): 0 ? 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=1): 0 1 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Force tagged (iter=2): 0 1 0 ? 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=2): 0 1 0 1 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Force tagged (iter=3): 0 1 0 1 0 ? 0 1 0 0 0 ? ? ? 0 ? 0 1 
Iteratively tagged (iter=3): 0 1 0 1 0 ? 0 1 0 0 0 ? ? 1 0 ? 0 1 
Force tagged (iter=4): 0 1 0 1 0 ? 0 1 0 0 0 ? ? 1 0 ? 0 1 
Iteratively tagged (iter=4): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 ? 0 1 
Force tagged (iter=5): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 ? 0 1 
Iteratively tagged (iter=5): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 
Force tagged (iter=6): 0 1 0 1 0 1 0 1 0 0 0 ? ? 1 0 1 0 1 


('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

debug=True이면 lcr 점수표와 띄어쓰기 교정이 되는 과정이 출력됩니다. 

In [11]:
model.correct(doc=sent, debug=True, force_abs_threshold=0.9, nonspace_threshold=nt, space_threshold=st, min_count=mc)

0: 이 (-1.000, 16)	lcr = (0.000, 0.000, -1.000)
1: 건 (0.750, 24)	lcr = (0.000, 0.750, 0.000)
2: 진 (-1.000, 53)	lcr = (0.000, -1.000, -1.000)
3: 짜 (0.703, 45)	lcr = (0.750, 0.655, 0.000)
4: 좋 (-1.000, 536)	lcr = (-1.000, 0.000, -1.000)
5: 은 (0.503, 1024)	lcr = (0.000, 0.377, 0.629)
6: 영 (-1.000, 1102)	lcr = (-1.000, -1.000, -1.000)
7: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
8: 라 (-0.998, 1015)	lcr = (-1.000, -1.000, -0.995)
9: 라 (-0.938, 1430)	lcr = (-0.882, -0.962, -0.968)
10: 랜 (-1.000, 1396)	lcr = (-1.000, -1.000, 0.000)
11: 드 (0.011, 920)	lcr = (0.011, 0.000, 0.000)
12: 진 (-1.000, 29)	lcr = (0.000, 0.000, -1.000)
13: 짜 (0.655, 29)	lcr = (0.000, 0.655, 0.000)
14: 좋 (-1.000, 536)	lcr = (-1.000, 0.000, -1.000)
15: 은 (0.502, 1022)	lcr = (0.000, 0.377, 0.627)
16: 영 (-1.000, 1022)	lcr = (-1.000, -1.000, 0.000)
17: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
force tagging i=0, score=-1.000
force tagging i=2, score=-1.000
force tagging i=4, score=-1.000
force tagging i=6, score=-1.000
force ta

('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])

### Load model and rules

이미 학습된 모델과 띄어쓰기가 반드시 이뤄져야 하는 rules를 아는 경우에 이를 로딩하여 이용할 수 있습니다. 

In [12]:
model = CountSpace()

model_fname = './tmp/space_model.json'
model.load_model(model_fname, json_format=True)

아래와 같이 모델을 중복으로 로딩할 수 있습니다. 중복으로 로딩될 때에는 (chars,tags)의 빈도수가 추가됩니다. 

In [13]:
another_model_fname = './tmp/space_model.json'
model.load_model(another_model_fname, json_format=True)

rules는 하나 이상의 파일이 입력될 수 있음을 고려하여 fname list로 받습니다. 

min_rule_length, max_rule_length는 rules에 이용되는 단어의 최소/최대 길이입니다. 

In [14]:
rule_fnames = ['./space_rules.txt']
min_rule_length = 2
max_rule_length = 3

rule_dict = RuleDict(min_rule_length, max_rule_length, rule_fnames)

In [15]:
rule_dict.rule_dict

{'가게는': (1, 0, 0, 1),
 '가게로': (1, 0, 0, 1),
 '가게야': (1, 0, 0, 1),
 '가게찌': (1, 0, 0, 1),
 '가령': (1, 0, 1),
 '가잣': (1, 0, 1),
 '가졍': (1, 0, 1),
 '가쥬': (1, 0, 1),
 '각각': (1, 0, 1)}

In [16]:
model.correct(doc=sent, verbose=verbose, force_abs_threshold=ft, nonspace_threshold=nt, space_threshold=st, min_count=mc)

('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])